# Intégration courbe de charge historique dans la bdd postgres

### récupération des données de puis l'API

In [ ]:
import os
from dotenv import load_dotenv
from app.api_client import get_consumption_load
from datetime import datetime, timedelta
import pandas as pd

# Charge les variables depuis le fichier .env
load_dotenv()

# Récupérer le token
API_TOKEN = os.getenv("API_TOKEN")
PRM = os.getenv("PRM")



start_date = "2024-12-01"

start_date_dt = datetime.strptime(start_date, "%Y-%m-%d")
end_date_dt   = start_date_dt + timedelta(days=7)
end_date = end_date_dt.strftime("%Y-%m-%d")
list_df_data = list()

while end_date_dt <= datetime.now() + timedelta(days=1):
    print(f"Récupération des données du {start_date} au {end_date}")
    data = get_consumption_load(PRM, start_date, end_date, API_TOKEN)
    start_date_dt = end_date_dt
    start_date = start_date_dt.strftime("%Y-%m-%d")
    end_date_dt   = start_date_dt + timedelta(days=7)
    end_date = end_date_dt.strftime("%Y-%m-%d")
    list_data = data['interval_reading']
    df = pd.DataFrame(list_data)
    list_df_data.append(df)

last_date = datetime.strptime(data['interval_reading'][-1]['date'], "%Y-%m-%d %H:%M:%S")

if last_date < datetime.now():
    start_date = last_date.strftime("%Y-%m-%d")
    end_date_dt   = last_date + timedelta(days=1)
    end_date = end_date_dt.strftime("%Y-%m-%d")
    data = get_consumption_load(PRM, start_date, end_date, API_TOKEN)
    list_data = data['interval_reading']
    df = pd.DataFrame(list_data)
    list_df_data.append(df)


dataconso = pd.concat(list_df_data, ignore_index=True)

# Convertir la colonne 'date' en datetime
dataconso['date'] = pd.to_datetime(dataconso['date'])

# Convertir 'value' en numérique (int ou float)
dataconso['value'] = pd.to_numeric(dataconso['value'])

# Définir l'index temporel
dataconso.set_index('date', inplace=True)

### intégration des données dans la bdd postgres

In [ ]:
import psycopg2
import pandas as pd
from psycopg2 import sql

# Connexion à PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="electricite",
    user="postgres",
    password="Labrax_007"
)

# Créer un curseur
cur = conn.cursor()

# Créer une table de test
cur.execute("""
CREATE TABLE IF NOT EXISTS courbecharge (
    id SERIAL PRIMARY KEY,
    timestamp TIMESTAMP,
    value FLOAT
)
""")

conn.commit() 



# Vérifie et convertis l'index si nécessaire
print("Type de l'index avant conversion :", type(dataconso.index))
dataconso.index = pd.to_datetime(dataconso.index)  # Force la conversion en DatetimeIndex
print("Type de l'index après conversion :", type(dataconso.index))

# Insertion des données
for index, row in dataconso.iterrows():
    try:
        cur.execute(
            sql.SQL("INSERT INTO courbecharge (timestamp, value) VALUES (%s, %s)"),
            (index, row['value'])
        )
    except Exception as e:
        print(f"Erreur à l'insertion de la ligne {index}: {e}")
        conn.rollback()
    else:
        conn.commit()

# Intégration des données journalières

## Récupération des données depuis le Drive

In [70]:
# lecture des données du 2021-12-22 to 2024-12-04
import pandas as pd
import os


dir_drive = "G:\Mon Drive\Consommation logement\données élec\jour"

file_1 = "enedis.xlsx"
df_histo1 = pd.read_excel(os.path.join(dir_drive, file_1))
#df_histo1['Horodate'] = pd.to_datetime(df_histo1['Horodate'])
df_histo1['consohp'] = df_histo1['EAS F1'].shift(-1) - df_histo1['EAS F1']
df_histo1['consohc'] = df_histo1['EAS F2'].shift(-1) - df_histo1['EAS F2'] 
df_histo1['conso_kwh'] = (df_histo1['consohp']+df_histo1['consohc'])/1000.0
df_histo1['jour'] = df_histo1['Horodate'].apply(lambda x: pd.to_datetime(x[0:10]))


In [ ]:
df_histo1

In [ ]:
import glob

# Chemin du répertoire (remplace par ton chemin)
dir_drive = "G:/Mon Drive/Consommation logement/données élec/jour/"

# Liste tous les fichiers Excel commençant par "ch1"
fichiers_excel = glob.glob(f"{dir_drive}02297250326360*.xlsx*")

list_df_data = list()
# Affiche les fichiers trouvés
for fichier in fichiers_excel:
    print(fichier)
    df_partielle = pd.read_excel(fichier, skiprows=13,sheet_name='Export Consommation Quotidienne', usecols="B,C")
    list_df_data.append(df_partielle)   

df_histo2 = pd.concat(list_df_data, ignore_index=True)
df_histo2['jour'] = pd.to_datetime(df_histo2['Date'], format="%d/%m/%Y")

df_histo2.sort_values(by='jour', inplace=True)  

df_histo2.columns = ['Date', 'conso_kwh', 'jour']
df_conso_histo = pd.concat([df_histo1[['conso_kwh', 'jour']], df_histo2[['conso_kwh', 'jour']]])
